In [ ]:
import os
import json, json5
import numpy as np
from tqdm import tqdm
import glob
import pandas as pd
import cprint

In [ ]:
all_classes = ['dam', 'storage-tank', 'ground-track-field', 'overpass', 'baseball-diamond', 'tennis-court', 'vehicle', 'basketball-court', 'golffield', 'harbor', 'expressway-service-area', 'chimney', 'trainstation', 'windmill', 'expressway-toll-station', 'ship', 'airport', 'bridge', 'airplane', 'stadium', 'soccer-ball-field', 'roundabout', 'swimming-pool', 'helicopter', 'container-crane', 'helipad']

exclude_phrases = ['flag', 'not provide', 'not specified', 'unknown', 'referred', 'referring',\
    'nose', 'vertical stabilizer', ' tail', 'tail ', 'facing', 'pointing',\
    'first-mentioned', 'aforementioned', 'previously mentioned', 'motion', 'day', 'night', 
]

correct_qa_types = ['object existence', 'object position', 'object quantity', 'object category', 'object color', 'object shape', 'object size', 'object direction', 'scene type', 'image', 'reasoning', 'rural or urban']

all_files = glob.glob('./Human_Check/Label_Your_Data_Kaust_DOTA_VAL_cleaned/*.json') + glob.glob('Human_Check/Label_Your_Data_Kaust_DIOR_VAL_cleaned/*.json')

all_files = [f for f in all_files if not f.endswith('input.json')]

len(all_files)

# Prepare Referring

In [15]:
all_items = []
valid_count = 0
non_valid = []

use_obb = False

for json_path in tqdm(sorted(all_files)):
    if json_path.endswith('input.json'):
        continue

    try:
        with open(json_path, 'r') as f:
            data = json5.load(f)

        if 'objects' in data:
            all_refs = data['objects']

        # item = {'id': json_path.split('.')[0], 'image': }
        conversations = []
        for ii, ref in enumerate(all_refs):
            if 'referring_sentence' not in ref or 'obj_corner' not in ref or 'obj_cls' not in ref:
                print('missing ref info', data['image'])
            
            x1,y1,x2,y2,x3,y3,x4,y4 = ref['obj_corner']
            # convert coordinates to float
            x1,y1,x2,y2,x3,y3,x4,y4 = map(float, [x1,y1,x2,y2,x3,y3,x4,y4])
            # horizontal bounding box
            xmin = max(min(x1,x2,x3,x4), 0)
            xmax = min(max(x1,x2,x3,x4), 1)
            ymin = max(min(y1,y2,y3,y4), 0)
            ymax = min(max(y1,y2,y3,y4), 1)
            # multiply by 100 to get integer
            xmin = int(xmin * 100)
            xmax = int(xmax * 100)
            ymin = int(ymin * 100)
            ymax = int(ymax * 100)

            obj_cls = ref['obj_cls'].lower()
            assert obj_cls in all_classes, obj_cls
            
            if use_obb:
                # calculate angble beased on the above four cornet points
                angle = np.arctan2(y2-y1, x2-x1) * 180 / np.pi
                bbox_str = "{" + f"<{x1}><{y1}><{x3}><{y3}>|<{angle}>" + "}"
            else:
                bbox_str = "{" + f"<{xmin}><{ymin}><{xmax}><{ymax}>" + "}"
            
            obj_size = ref['obj_size']
            
            item_dict = {
                "image_id": data['image'], 
                "question": ref['referring_sentence'], 
                "ground_truth": bbox_str, 
                "dataset": "RSBench", 
                "question_id": valid_count, 
                "type": "ref", 
                "unique": ref['is_unique'],
                "obj_corner": ref['obj_corner'],
                "obj_cls": obj_cls,
                "obj_ids": [0, ], 
                "size_group": obj_size,
            }
            all_items.append(item_dict)
        
            valid_count += 1

    except Exception as e:
        print('skipping', json_path, 'error', e)
        non_valid.append(json_path)
        continue

with open('RSBench_EVAL_referring_v2.json', 'w') as f:
    json.dump(all_items, f, indent=4)

print('valid_count', valid_count, len(all_files), len(non_valid))

100%|██████████| 9350/9350 [04:05<00:00, 38.16it/s]


valid_count 16159 9350 0


In [16]:
import inflect
convert = inflect.engine()
all_numbers = [convert.number_to_words(x) for x in range(100)]


# VQA

In [18]:

all_items = []
valid_count = 0
non_valid = []

use_obb = False
skip_qas = 0
for json_path in tqdm(sorted(all_files)):
    if json_path.endswith('input.json'):
        continue

    try:
        with open(json_path, 'r') as f:
            data = json5.load(f)

        if 'qa_pairs' in data:
            all_qas = data['qa_pairs']

        for jj, qa in enumerate(all_qas):
            if 'question' not in qa or 'answer' not in qa or 'type' not in qa:
                print('missing qa info', data['image'])
                continue
            
            skip = False
            for phr in exclude_phrases + ['source', 'resolution']:
                if phr in qa['answer'] or phr in qa['question']:
                    print('skipping qa', json_path, 'due to', phr)
                    skip = True
                    skip_qas += 1
                    break
            if skip: continue
            
            skip = False
            if 'which' in qa['question'].lower():
                for num in range(100):
                    if str(num) in qa['answer'] or all_numbers[num] in qa['answer']:
                        skip = True
                        skip_qas += 1
                        print('question by object order, '+ qa['answer'])
                        break
            if skip: continue

            qa['type'] = qa['type'].strip().lower()
            assert qa['type'] in correct_qa_types, qa['type']

            item_dict = {
                "image_id": data['image'], 
                "question": qa['question'], 
                "ground_truth": qa['answer'], 
                "dataset": "RSBench", 
                "question_id": valid_count, 
                "type": qa['type'], 
            }
            all_items.append(item_dict)
        
            valid_count += 1

    except Exception as e:
        print('skipping', json_path, 'error', e)
        non_valid.append(json_path)
        continue

with open('RSBench_EVAL_vqa_v2.json', 'w') as f:
    json.dump(all_items, f, indent=4)

print('valid_count', valid_count, len(all_files), len(non_valid), skip_qas)

100%|██████████| 9350/9350 [04:01<00:00, 38.69it/s]


valid_count 37409 9350 0 0


In [19]:
valid_count, skip_qas

(37409, 0)

# Captioning

In [20]:

all_items = []
valid_count = 0
non_valid = []

use_obb = False

for json_path in tqdm(sorted(all_files)):
    if json_path.endswith('input.json'):
        continue

    try:
        with open(json_path, 'r') as f:
            data = json5.load(f)

        if 'caption' in data:
            caption = data['caption']
            item_dict = {
                "image_id": data['image'], 
                "ground_truth": caption, 
                "question": "Describe the image in detail",
                "dataset": "RSBench", 
                "question_id": valid_count, 
                "type": "caption", 
            }
            all_items.append(item_dict)
        
            valid_count += 1

    except Exception as e:
        print('skipping', json_path, 'error', e)
        non_valid.append(json_path)
        continue

with open('RSBench_EVAL_Cap_v2.json', 'w') as f:
    json.dump(all_items, f, indent=4)

print('valid_count', valid_count, len(all_files), len(non_valid))

100%|██████████| 9350/9350 [04:02<00:00, 38.49it/s]


valid_count 9350 9350 0
